## import data

In [1]:
## import data
import numpy as np
import pandas as pd

data = pd.read_csv('pheno_allgenic.csv')
data_prepost = data[pd.notnull(data['SRS.prepost.cat'])]

x_prepost = data_prepost.iloc[:, list(range(3, 30))]
y_prepost = data_prepost.iloc[:, 1]   
data_prepost.head()

,SAMPLE,SRS.prepost.cat,SRS.prefu.cat,SRS.pre.p,ddcgas.pre,cgi.pre,abas.f.total.pre,ados.tot,wisc.fsiq,comorbid.adhd,...,cnvgene_num,BIN3_all_recode,clnsig_recode,X0.5asd,X1.0adhd,lof_cln,synapgene_ratio_comb,synapgene_asd0.01_common,damage_num,ndd_num
0,4514543,0.0,1.0,59,53.0,4.0,440.0,17,115,1,...,0,0,0,-0.001273,-0.001123,0.0,0.000000,0.089458,57.0,9.0
1,5008214,1.0,NaN,91,62.0,5.0,295.0,17,108,1,...,0,0,0,-0.001118,-0.000905,0.0,0.022727,0.086022,57.0,5.0
2,4925137,0.0,NaN,135,55.0,4.0,313.0,9,95,1,...,2,0,0,-0.001059,-0.000967,0.0,0.105263,0.089085,47.0,2.0
3,4799730,0.0,1.0,114,65.0,4.0,394.0,14,99,1,...,2,0,0,-0.001203,-0.000920,0.0,0.054795,0.086584,83.0,12.0
4,4904181,0.0,0.0,72,56.0,4.0,375.0,11,74,0,...,27,1,1,-0.001272,-0.001072,0.0,0.070175,0.082000,44.0,5.0


In [2]:
data_prepost.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186 entries, 0 to 206
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SAMPLE                    186 non-null    int64  
 1   SRS.prepost.cat           186 non-null    float64
 2   SRS.prefu.cat             161 non-null    float64
 3   SRS.pre.p                 186 non-null    int64  
 4   ddcgas.pre                184 non-null    float64
 5   cgi.pre                   184 non-null    float64
 6   abas.f.total.pre          185 non-null    float64
 7   ados.tot                  186 non-null    int64  
 8   wisc.fsiq                 186 non-null    int64  
 9   comorbid.adhd             186 non-null    int64  
 10  comorbid.anxiety          186 non-null    int64  
 11  comorbid.depression       186 non-null    int64  
 12  comorbid.other            186 non-null    int64  
 13  age.pre                   186 non-null    int64  
 14  Gender    

In [3]:
## change binary features type
cat_cols = ['comorbid.adhd', 'comorbid.anxiety', 'comorbid.depression','comorbid.other', 'Gender','treatment.kontakt','treatment.pharma','treatment.councel','treatment.cbt','BIN3_all_recode','clnsig_recode','lof_cln']
for i in cat_cols:
    x_prepost[i] = x_prepost[i].astype('object')
x_prepost['cnv_size'] = x_prepost['cnv_size'].astype('int')
x_prepost.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186 entries, 0 to 206
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SRS.pre.p                 186 non-null    int64  
 1   ddcgas.pre                184 non-null    float64
 2   cgi.pre                   184 non-null    float64
 3   abas.f.total.pre          185 non-null    float64
 4   ados.tot                  186 non-null    int64  
 5   wisc.fsiq                 186 non-null    int64  
 6   comorbid.adhd             186 non-null    object 
 7   comorbid.anxiety          186 non-null    object 
 8   comorbid.depression       186 non-null    object 
 9   comorbid.other            186 non-null    object 
 10  age.pre                   186 non-null    int64  
 11  Gender                    186 non-null    object 
 12  treatment.kontakt         186 non-null    object 
 13  treatment.pharma          179 non-null    object 
 14  treatment.

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [14]:
## imputation missing values using K nearest neighbors
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)

x_prepost_impute = pd.DataFrame(imputer.fit_transform(x_prepost), columns = x_prepost.columns)
int_cols = ['SRS.pre.p', 'ddcgas.pre', 'cgi.pre', 'abas.f.total.pre', 'ados.tot', 'wisc.fsiq', 'comorbid.adhd', 'comorbid.anxiety', 'comorbid.depression','comorbid.other', 'age.pre', 'Gender', 'treatment.kontakt', 'treatment.pharma', 'treatment.councel', 'treatment.cbt', 'cnv_size', 'cnvgene_num', 'BIN3_all_recode', 'clnsig_recode', 'lof_cln', 'damage_num', 'ndd_num']
for i in int_cols:
    x_prepost_impute[i] = x_prepost_impute[i].astype(int)
x_prepost_impute.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SRS.pre.p                 186 non-null    int64  
 1   ddcgas.pre                186 non-null    int64  
 2   cgi.pre                   186 non-null    int64  
 3   abas.f.total.pre          186 non-null    int64  
 4   ados.tot                  186 non-null    int64  
 5   wisc.fsiq                 186 non-null    int64  
 6   comorbid.adhd             186 non-null    int64  
 7   comorbid.anxiety          186 non-null    int64  
 8   comorbid.depression       186 non-null    int64  
 9   comorbid.other            186 non-null    int64  
 10  age.pre                   186 non-null    int64  
 11  Gender                    186 non-null    int64  
 12  treatment.kontakt         186 non-null    int64  
 13  treatment.pharma          186 non-null    int64  
 14  treatment.

In [15]:
x_prepost_impute.describe()

,SRS.pre.p,ddcgas.pre,cgi.pre,abas.f.total.pre,ados.tot,wisc.fsiq,comorbid.adhd,comorbid.anxiety,comorbid.depression,comorbid.other,...,cnvgene_num,BIN3_all_recode,clnsig_recode,X0.5asd,X1.0adhd,lof_cln,synapgene_ratio_comb,synapgene_asd0.01_common,damage_num,ndd_num
count,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,...,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000
mean,85.591398,57.774194,4.322581,376.446237,10.779570,98.387097,0.731183,0.236559,0.123656,0.145161,...,6.784946,0.064516,0.086022,-0.001229,-0.001059,0.204301,0.047584,0.089554,54.639785,4.725806
std,24.536013,7.151834,0.807549,63.434408,3.405847,13.019762,0.444541,0.426116,0.330077,0.353214,...,47.436865,0.246333,0.281153,0.000083,0.000066,0.510625,0.031254,0.003968,17.050777,2.466002
min,28.000000,41.000000,2.000000,223.000000,6.000000,70.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.001497,-0.001218,0.000000,0.000000,0.081235,29.000000,0.000000
25%,70.000000,52.000000,4.000000,333.250000,8.000000,89.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.001281,-0.001096,0.000000,0.025398,0.086512,45.000000,3.000000
50%,86.500000,58.000000,4.000000,378.000000,10.000000,99.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.001236,-0.001058,0.000000,0.042553,0.088976,50.000000,5.000000
75%,101.000000,62.000000,5.000000,418.750000,13.000000,108.000000,1.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,-0.001171,-0.001023,0.000000,0.066667,0.092479,59.750000,6.000000
max,142.000000,82.000000,6.000000,539.000000,20.000000,128.000000,1.000000,1.000000,1.000000,1.000000,...,617.000000,1.000000,1.000000,-0.000997,-0.000900,2.000000,0.184211,0.099072,153.000000,13.000000


In [16]:
## data center and scale
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_prepost_trans = scaler.fit_transform(x_prepost_impute)
pd.DataFrame(x_prepost_trans).describe()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
count,1.860000e+02,1.860000e+02,1.860000e+02,1.860000e+02,1.860000e+02,1.860000e+02,1.860000e+02,1.860000e+02,1.860000e+02,1.860000e+02,...,1.860000e+02,1.860000e+02,1.860000e+02,1.860000e+02,1.860000e+02,1.860000e+02,1.860000e+02,1.860000e+02,1.860000e+02,1.860000e+02
mean,-7.088117e-17,8.654964e-17,3.682837e-16,1.575800e-16,6.938894e-17,-2.638272e-16,-1.468359e-16,-7.640244e-17,-9.341393e-17,-5.670494e-17,...,4.357327e-17,-1.241540e-16,-1.289291e-16,-4.834842e-17,1.225125e-15,-2.387576e-16,2.104052e-16,-1.726274e-15,-4.655774e-17,1.134099e-16
std,1.002699e+00,1.002699e+00,1.002699e+00,1.002699e+00,1.002699e+00,1.002699e+00,1.002699e+00,1.002699e+00,1.002699e+00,1.002699e+00,...,1.002699e+00,1.002699e+00,1.002699e+00,1.002699e+00,1.002699e+00,1.002699e+00,1.002699e+00,1.002699e+00,1.002699e+00,1.002699e+00
min,-2.353554e+00,-2.351770e+00,-2.883848e+00,-2.425504e+00,-1.407130e+00,-2.186193e+00,-1.649242e+00,-5.566499e-01,-3.756385e-01,-4.120817e-01,...,-1.434171e-01,-2.626129e-01,-3.067860e-01,-3.242498e+00,-2.395599e+00,-4.011797e-01,-1.526617e+00,-2.102065e+00,-1.507790e+00,-1.921556e+00
25%,-6.371646e-01,-8.095515e-01,-4.005344e-01,-6.827970e-01,-8.183197e-01,-7.229343e-01,-1.649242e+00,-5.566499e-01,-3.756385e-01,-4.120817e-01,...,-1.434171e-01,-2.626129e-01,-3.067860e-01,-6.275301e-01,-5.541824e-01,-4.011797e-01,-7.117942e-01,-7.686247e-01,-5.668834e-01,-7.017286e-01
50%,3.713132e-02,3.165844e-02,-4.005344e-01,2.456013e-02,-2.295094e-01,4.720190e-02,6.063391e-01,-5.566499e-01,-3.756385e-01,-4.120817e-01,...,-1.434171e-01,-2.626129e-01,-3.067860e-01,-8.348654e-02,1.440047e-02,-4.011797e-01,-1.613969e-01,-1.461592e-01,-2.728502e-01,1.114896e-01
75%,6.296944e-01,5.924651e-01,8.411222e-01,6.686898e-01,6.537061e-01,7.403245e-01,6.063391e-01,-5.566499e-01,-3.756385e-01,-4.120817e-01,...,-1.222796e-01,-2.626129e-01,-3.067860e-01,6.926741e-01,5.498540e-01,-4.011797e-01,6.122279e-01,7.389722e-01,3.005146e-01,5.180987e-01
max,2.305218e+00,3.396498e+00,2.082779e+00,2.569465e+00,2.714542e+00,2.280597e+00,6.063391e-01,1.796461e+00,2.662134e+00,2.426703e+00,...,1.289845e+01,3.807887e+00,3.259601e+00,2.790984e+00,2.403649e+00,3.526159e+00,4.383349e+00,2.404923e+00,5.784235e+00,3.364362e+00


## 1 support vector machine with rbf kernel

In [17]:
## try rbf SVM model for training. For feature selection, use sequential forward selection method to select most important feature set in each inner CV and calculate the total number of each features after 10-time of outer CV.
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold 
from sklearn.svm import SVC
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
    
y_prepost_array = y_prepost.to_numpy()
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

feature_list = []
for train, test in outer_cv.split(x_prepost_trans, y_prepost_array):
    #rbf SVC
    clf_outer = SVC(class_weight='balanced', kernel='rbf', random_state=0)  
    ##stepwise forward selection
    sfs = SFS(clf_outer, k_features=(1, 27), forward=True, floating=True, verbose=0, scoring='roc_auc',  cv=inner_cv)   
    sfs = sfs.fit(x_prepost_trans[train], y_prepost_array[train])
    feature_list.append(list(sfs.k_feature_idx_))


In [18]:
## summary the total select times for each feature after 10-time outer CV as importance measurement
feature_sum_dict = {}
feature_sum_sort_dict = {}
for i in feature_list:
    for j in i:
        if x_prepost.columns[j] not in feature_sum_dict:
            feature_sum_dict[x_prepost.columns[j]] = 1
        else:
            feature_sum_dict[x_prepost.columns[j]] += 1

## sort features in feature dict based on selected times
feature_sum_sort_dict = {k: v for k, v in sorted(feature_sum_dict.items(), key=lambda item: item[1], reverse=True)}
feature_sum_sort_dict

{'synapgene_ratio_comb': 8,
 'synapgene_asd0.01_common': 7,
 'treatment.kontakt': 7,
 'treatment.cbt': 5,
 'ndd_num': 5,
 'lof_cln': 5,
 'ados.tot': 4,
 'comorbid.anxiety': 4,
 'comorbid.other': 4,
 'cnv_size': 4,
 'BIN3_all_recode': 4,
 'age.pre': 3,
 'SRS.pre.p': 3,
 'cnvgene_num': 3,
 'X1.0adhd': 2,
 'damage_num': 2,
 'abas.f.total.pre': 2,
 'treatment.councel': 2,
 'wisc.fsiq': 1,
 'comorbid.depression': 1,
 'X0.5asd': 1,
 'ddcgas.pre': 1,
 'treatment.pharma': 1}

In [29]:
## choose the most selected features in SVM for prediction. Use gridsearchCV to optimize parameters
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_curve, auc  

y_prepost_array = y_prepost.to_numpy()
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

fpr_train_dict = {}
tpr_train_dict = {}
fpr_test_dict = {}
tpr_test_dict = {}
auc_train_dict = {}
auc_test_dict = {}
accuracy_train_dict = {}
accuracy_test_dict = {}

c_dict = {}
gamma_dict = {}
feature_totalselect_dict = {}
scaler = StandardScaler()

# rbf SVC
tuned_parameters = [{'C': [0.001, 0.01, 0.1, 1], 'gamma': [0.001, 0.01, 0.1, 1]}] 
svm = SVC(kernel='rbf', class_weight='balanced')
for i in range(2,8):   ##chose feature sets that have total selected times from > 2 to > 7
    feature_list = []
    feature_select_dict = {}
    feature_select_dict={a:u for a, u in feature_sum_sort_dict.items() if u > i}
    for b in feature_select_dict:
        feature_list.append(list(x_prepost.columns).index(b)) 
    x_prepost_sel = x_prepost_impute.iloc[:, feature_list]
    x_prepost_trans_sel = scaler.fit_transform(x_prepost_sel)
    
    c_dict['feature select time > %i'%i]=[]
    gamma_dict['feature select time > %i'%i]=[]
        
    fpr_train_dict['feature select time > %i'%i]=[]  ## fpr: sensitivity
    tpr_train_dict['feature select time > %i'%i]=[]  ## 1-tpr: specificity
    fpr_test_dict['feature select time > %i'%i]=[]
    tpr_test_dict['feature select time > %i'%i]=[]
    auc_train_dict['feature select time > %i'%i]=[]
    auc_test_dict['feature select time > %i'%i]=[]
    accuracy_train_dict['feature select time > %i'%i]=[]
    accuracy_test_dict['feature select time > %i'%i]=[]
    feature_totalselect_dict['feature select time > %i'%i] = list(feature_select_dict.keys())
        
    for train, test in outer_cv.split(x_prepost_trans_sel, y_prepost_array):
        clf = GridSearchCV(estimator=svm, param_grid=tuned_parameters, scoring='roc_auc', cv=inner_cv)
        clf.fit(x_prepost_trans[train], y_prepost_array[train])
        #rbf
        clf_outer = SVC(class_weight='balanced', C=clf.best_params_['C'], kernel='rbf', gamma=clf.best_params_['gamma'], random_state=0)  
        
        clf_outer.fit(x_prepost_trans[train], y_prepost_array[train])
        predicted, y_pred = clf_outer.predict(x_prepost_trans[train]), clf_outer.predict(x_prepost_trans[test])

        fpr_train, tpr_train, thresholds_train = roc_curve(y_prepost_array[train], predicted)
        fpr_test, tpr_test, thresholds_test = roc_curve(y_prepost_array[test], y_pred)
        auc_train, auc_test = auc(fpr_train, tpr_train), auc(fpr_test, tpr_test)
        accuracy_train, accuracy_test = accuracy_score(y_prepost_array[train], predicted), accuracy_score(y_prepost_array[test], y_pred)

        c_dict['feature select time > %i'%i].append(clf.best_params_['C'])
        gamma_dict['feature select time > %i'%i].append(clf.best_params_['gamma'])
        
        fpr_train_dict['feature select time > %i'%i].append(fpr_train[1])
        tpr_train_dict['feature select time > %i'%i].append(tpr_train[1])
        fpr_test_dict['feature select time > %i'%i].append(fpr_test[1])
        tpr_test_dict['feature select time > %i'%i].append(tpr_test[1])
        auc_train_dict['feature select time > %i'%i].append(auc_train)
        auc_test_dict['feature select time > %i'%i].append(auc_test)
        accuracy_train_dict['feature select time > %i'%i].append(accuracy_train)
        accuracy_test_dict['feature select time > %i'%i].append(accuracy_test)

In [30]:
## calculate average auc train and test value in outer CV
auc_avg_test_dict = {}
auc_avg_train_dict = {}
for key in auc_test_dict:
    auc_test, auc_train = 0.0, 0.0
    for time in auc_test_dict[key]:
        auc_test += time
    for time in auc_train_dict[key]:
        auc_train += time
    auc_avg_test, auc_avg_train = auc_test/10, auc_train/10
    auc_avg_test_dict[key]=auc_avg_test
    auc_avg_train_dict[key]=auc_avg_train
print(auc_avg_test_dict)


{'feature select time > 2': 0.49777777777777776, 'feature select time > 3': 0.49777777777777776, 'feature select time > 4': 0.49777777777777776, 'feature select time > 5': 0.49777777777777776, 'feature select time > 6': 0.49777777777777776, 'feature select time > 7': 0.49777777777777776}


In [31]:
print(auc_avg_train_dict)

{'feature select time > 2': 0.5536221085840628, 'feature select time > 3': 0.5536221085840628, 'feature select time > 4': 0.5536221085840628, 'feature select time > 5': 0.5536221085840628, 'feature select time > 6': 0.5536221085840628, 'feature select time > 7': 0.5536221085840628}


## 2 Logistic regression with elastic net 

In [34]:
## feature selection 

from sklearn.model_selection import StratifiedKFold  
from sklearn.linear_model import LogisticRegressionCV

y_prepost_array = y_prepost.to_numpy()
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

feature_dict = {}
# logistic regression
i=0
for train, test in outer_cv.split(x_prepost_trans, y_prepost_array):
    i+=1
    clf = LogisticRegressionCV(penalty='elasticnet', solver='saga', random_state=0, class_weight='balanced', cv=inner_cv, l1_ratios=[0, 0.2, 0.4, 0.6, 0.8, 1]) 
    clf_fit = clf.fit(x_prepost_trans[train], y_prepost_array[train])
    feature_dict['round %i' %i] = clf_fit.coef_ 

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

...

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

In [35]:
## average feature importance after cross validation
feature_array=0
for i in range(1,11):
    feature_array += feature_dict['round %i' %i] 

feature_sum_dict = {}
for i in range(0,27):
    feature_sum_dict[x_prepost.columns[i]] = abs(feature_array[0,i]/10)
    
feature_sum_sort_dict = {k: v for k, v in sorted(feature_sum_dict.items(), key=lambda item: item[1], reverse=True)}  
feature_sum_sort_dict

{'comorbid.anxiety': 0.22274861727649592,
 'comorbid.adhd': 0.17394448712581287,
 'comorbid.depression': 0.16338155923772135,
 'cnvgene_num': 0.14121468813253,
 'ndd_num': 0.12125537979802,
 'synapgene_ratio_comb': 0.10521852669355887,
 'SRS.pre.p': 0.1048006662350949,
 'cgi.pre': 0.0983897087773494,
 'treatment.kontakt': 0.09690266777158987,
 'cnv_size': 0.09628356038289625,
 'damage_num': 0.08424803614415226,
 'comorbid.other': 0.0816994339880929,
 'age.pre': 0.06495766047589667,
 'synapgene_asd0.01_common': 0.058707027343525134,
 'lof_cln': 0.05012188259593843,
 'ados.tot': 0.048757052670065915,
 'X1.0adhd': 0.036295384477474255,
 'abas.f.total.pre': 0.03574283369692294,
 'ddcgas.pre': 0.03042751907422526,
 'X0.5asd': 0.029443319380740806,
 'treatment.cbt': 0.020547271304246436,
 'wisc.fsiq': 0.016076422308480786,
 'clnsig_recode': 0.015018198839603123,
 'treatment.councel': 0.007319839397555263,
 'treatment.pharma': 0.005925132184049416,
 'Gender': 0.0023191147015733256,
 'BIN3_all

In [36]:
## use selected features to train the model
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_curve, auc  
from sklearn.linear_model import LogisticRegressionCV

y_prepost_array = y_prepost.to_numpy()
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

fpr_train_dict = {}
tpr_train_dict = {}
fpr_test_dict = {}
tpr_test_dict = {}
auc_train_dict = {}
auc_test_dict = {}
accuracy_train_dict = {}
accuracy_test_dict = {}
c_dict = {}
l1_dict = {}

feature_totalselect_dict = {}
scaler = StandardScaler()

# logistic regression
for i in range(2,20):   
    feature_list = []
    feature_select_dict = {}
    feature_select_dict={a:u for a, u in feature_sum_sort_dict.items() if list(feature_sum_sort_dict.keys()).index(a) <= i}
    for b in feature_select_dict:
        feature_list.append(list(x_prepost.columns).index(b)) 
    x_prepost_sel = x_prepost_impute.iloc[:, feature_list]
    x_prepost_trans_sel = scaler.fit_transform(x_prepost_sel)
    
    fpr_train_dict['%i selected features'%i]=[]
    tpr_train_dict['%i selected features'%i]=[]
    fpr_test_dict['%i selected features'%i]=[]
    tpr_test_dict['%i selected features'%i]=[]
    auc_train_dict['%i selected features'%i]=[]
    auc_test_dict['%i selected features'%i]=[]
    accuracy_train_dict['%i selected features'%i]=[]
    accuracy_test_dict['%i selected features'%i]=[]
    feature_totalselect_dict['%i selected features'%i] = list(feature_select_dict.keys())
    
    for train, test in outer_cv.split(x_prepost_trans_sel, y_prepost_array):
        clf = LogisticRegressionCV(penalty='elasticnet', solver='saga', random_state=0, class_weight='balanced', cv=inner_cv, Cs=[0.001, 0.01, 0.1, 1, 10], l1_ratios=[0, 0.2, 0.4, 0.6, 0.8, 1])  
        clf_fit = clf.fit(x_prepost_trans_sel[train], y_prepost_array[train])
        
        predicted, y_pred = clf_fit.predict(x_prepost_trans_sel[train]), clf_fit.predict(x_prepost_trans_sel[test])
        fpr_train, tpr_train, thresholds_train = roc_curve(y_prepost_array[train], predicted)
        fpr_test, tpr_test, thresholds_test = roc_curve(y_prepost_array[test], y_pred)
        auc_train, auc_test = auc(fpr_train, tpr_train), auc(fpr_test, tpr_test)
        accuracy_train, accuracy_test = accuracy_score(y_prepost_array[train], predicted), accuracy_score(y_prepost_array[test], y_pred)
        
        c_dict['%i selected features'%i] = clf_fit.C_
        l1_dict['%i selected features'%i] = clf_fit.l1_ratio_
        
        fpr_train_dict['%i selected features'%i].append(fpr_train[1])
        tpr_train_dict['%i selected features'%i].append(tpr_train[1])
        fpr_test_dict['%i selected features'%i].append(fpr_test[1])
        tpr_test_dict['%i selected features'%i].append(tpr_test[1])
        auc_train_dict['%i selected features'%i].append(auc_train)
        auc_test_dict['%i selected features'%i].append(auc_test)
        accuracy_train_dict['%i selected features'%i].append(accuracy_train)
        accuracy_test_dict['%i selected features'%i].append(accuracy_test)

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

...

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: Convergence

In [37]:
## calculate average auc train and test value in outer CV
auc_avg_test_dict = {}
auc_avg_train_dict = {}
for key in auc_test_dict:
    auc_test, auc_train = 0.0, 0.0
    for time in auc_test_dict[key]:
        auc_test += time
    for time in auc_train_dict[key]:
        auc_train += time
    auc_avg_test, auc_avg_train = auc_test/10, auc_train/10
    auc_avg_test_dict[key]=auc_avg_test
    auc_avg_train_dict[key]=auc_avg_train
print(auc_avg_test_dict)

{'2 selected features': 0.48388888888888887, '3 selected features': 0.48388888888888887, '4 selected features': 0.48, '5 selected features': 0.5104166666666667, '6 selected features': 0.5729166666666667, '7 selected features': 0.5658333333333333, '8 selected features': 0.5669444444444445, '9 selected features': 0.5719444444444444, '10 selected features': 0.59875, '11 selected features': 0.6222222222222221, '12 selected features': 0.5475, '13 selected features': 0.5372222222222224, '14 selected features': 0.5322222222222223, '15 selected features': 0.5033333333333333, '16 selected features': 0.5051388888888889, '17 selected features': 0.5229166666666666, '18 selected features': 0.5177777777777779, '19 selected features': 0.50875}


In [38]:
print(auc_avg_train_dict)

{'2 selected features': 0.5367152690681527, '3 selected features': 0.5367152690681527, '4 selected features': 0.5443749353512367, '5 selected features': 0.5749886460609509, '6 selected features': 0.6093610358671339, '7 selected features': 0.6112048428388566, '8 selected features': 0.6288894069768944, '9 selected features': 0.6261354343103916, '10 selected features': 0.6374235528374342, '11 selected features': 0.647788284997608, '12 selected features': 0.6107971354132996, '13 selected features': 0.6106200301263236, '14 selected features': 0.5773269110174422, '15 selected features': 0.5975445672411788, '16 selected features': 0.5876265580351948, '17 selected features': 0.599103564086319, '18 selected features': 0.5879919205208105, '19 selected features': 0.593231153269288}


## 3 linear SVM 

In [45]:
## RFE CV for feature selection
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold  
from sklearn.svm import LinearSVC

y_prepost_array = y_prepost.to_numpy()
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

feature_dict = {}

i=0 
for train, test in outer_cv.split(x_prepost_trans, y_prepost_array):
    i+=1
    estimator = LinearSVC(random_state=0, class_weight='balanced')  
    selector = RFECV(estimator, step=1, cv=inner_cv, scoring = 'roc_auc')
    selector = selector.fit(x_prepost_trans[train], y_prepost_array[train])
    feature_dict['round %i' %i] = selector.ranking_  #Selected features are assigned as rank 1.    

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, i

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, i

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, i

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, i

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, i

...

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, i

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, i

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, i

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, i

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, i

In [46]:
## summary feature selected times
feature_sum_dict = {}
feature_sum_sort_dict = {}
for i in range(0, len(x_prepost.columns)):
    feature_sum_dict[x_prepost.columns[i]] = 0
    for j in range(1,11):
        if feature_dict['round %d' %j] [i] == 1:
            feature_sum_dict[x_prepost.columns[i]] +=1
feature_sum_sort_dict = {k: v for k, v in sorted(feature_sum_dict.items(), key=lambda item: item[1], reverse=True)} 
feature_sum_sort_dict

{'comorbid.anxiety': 5,
 'comorbid.depression': 5,
 'synapgene_ratio_comb': 5,
 'SRS.pre.p': 3,
 'comorbid.adhd': 3,
 'treatment.kontakt': 3,
 'cnv_size': 3,
 'cgi.pre': 2,
 'comorbid.other': 2,
 'cnvgene_num': 2,
 'damage_num': 2,
 'ndd_num': 2,
 'ddcgas.pre': 1,
 'abas.f.total.pre': 1,
 'ados.tot': 1,
 'wisc.fsiq': 1,
 'age.pre': 1,
 'clnsig_recode': 1,
 'X0.5asd': 1,
 'X1.0adhd': 1,
 'lof_cln': 1,
 'synapgene_asd0.01_common': 1,
 'Gender': 0,
 'treatment.pharma': 0,
 'treatment.councel': 0,
 'treatment.cbt': 0,
 'BIN3_all_recode': 0}

In [47]:
## use selected features to train the model
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_curve, auc  
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

y_prepost_array = y_prepost.to_numpy()
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

fpr_train_dict = {}
tpr_train_dict = {}
fpr_test_dict = {}
tpr_test_dict = {}
auc_train_dict = {}
auc_test_dict = {}
accuracy_train_dict = {}
accuracy_test_dict = {}
c_dict = {}

feature_totalselect_dict = {}

scaler = StandardScaler()
svm = LinearSVC(random_state=0, class_weight='balanced')
tuned_parameters = [{'C': [0.001, 0.01, 0.1, 1, 10]}] 
#linear SVM
for i in range(0, 5):  
    feature_list = []
    feature_select_dict = {}
    feature_select_dict={a:u for a, u in feature_sum_sort_dict.items() if u > i}
    for b in feature_select_dict:
        feature_list.append(list(x_prepost.columns).index(b)) 
    x_prepost_sel = x_prepost_impute.iloc[:, feature_list]
    x_prepost_trans_sel = scaler.fit_transform(x_prepost_sel)

    c_dict['feature select time > %i'%i]=[]
    fpr_train_dict['feature select time > %i'%i]=[]
    tpr_train_dict['feature select time > %i'%i]=[]
    fpr_test_dict['feature select time > %i'%i]=[]
    tpr_test_dict['feature select time > %i'%i]=[]
    auc_train_dict['feature select time > %i'%i]=[]
    auc_test_dict['feature select time > %i'%i]=[]
    accuracy_train_dict['feature select time > %i'%i]=[]
    accuracy_test_dict['feature select time > %i'%i]=[]
    feature_totalselect_dict['feature select time > %i'%i] = list(feature_select_dict.keys())
    
    for train, test in outer_cv.split(x_prepost_trans_sel, y_prepost_array):
        clf = GridSearchCV(estimator=svm, param_grid=tuned_parameters, scoring='roc_auc', cv=inner_cv) 
        clf.fit(x_prepost_trans_sel[train], y_prepost_array[train])
        clf_outer = LinearSVC(penalty='l2',random_state=0, class_weight='balanced', C=clf.best_params_['C'])  
        clf_outer.fit(x_prepost_trans_sel[train], y_prepost_array[train])
        
        predicted, y_pred = clf_outer.predict(x_prepost_trans_sel[train]), clf_outer.predict(x_prepost_trans_sel[test])
        fpr_train, tpr_train, thresholds_train = roc_curve(y_prepost_array[train], predicted)
        fpr_test, tpr_test, thresholds_test = roc_curve(y_prepost_array[test], y_pred)
        auc_train, auc_test = auc(fpr_train, tpr_train), auc(fpr_test, tpr_test)
        accuracy_train, accuracy_test = accuracy_score(y_prepost_array[train], predicted), accuracy_score(y_prepost_array[test], y_pred)
        
        c_dict['feature select time > %i'%i].append(clf.best_params_['C'])

        fpr_train_dict['feature select time > %i'%i].append(fpr_train[1])
        tpr_train_dict['feature select time > %i'%i].append(tpr_train[1])
        fpr_test_dict['feature select time > %i'%i].append(fpr_test[1])
        tpr_test_dict['feature select time > %i'%i].append(tpr_test[1])
        auc_train_dict['feature select time > %i'%i].append(auc_train)
        auc_test_dict['feature select time > %i'%i].append(auc_test)
        accuracy_train_dict['feature select time > %i'%i].append(accuracy_train)
        accuracy_test_dict['feature select time > %i'%i].append(accuracy_test)

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, i

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, i

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, i

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, i

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, i

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, i

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, i

/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danli/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, i

In [48]:
## calculate average auc train and test value in outer CV
auc_avg_test_dict = {}
auc_avg_train_dict = {}
accuracy_avg_test_dict = {}
accuracy_avg_train_dict = {}
sensi_avg_test_dict = {}
sensi_avg_train_dict = {}
speci_avg_test_dict = {}
speci_avg_train_dict = {}
for key in auc_test_dict:
    auc_test, auc_train = 0.0, 0.0
    accuracy_test, accuracy_train = 0.0, 0.0
    sensi_test, sensi_train = 0.0, 0.0
    speci_test, speci_train = 0.0, 0.0
    for time in auc_test_dict[key]:
        auc_test += time
    for time in auc_train_dict[key]:
        auc_train += time
    for time in accuracy_test_dict[key]:
        accuracy_test += time
    for time in accuracy_train_dict[key]:
        accuracy_train += time
    for time in tpr_test_dict[key]:
        sensi_test += time
    for time in tpr_train_dict[key]:
        sensi_train += time
    for time in fpr_test_dict[key]:
        speci_test += 1-time
    for time in fpr_train_dict[key]:
        speci_train += 1-time
    auc_avg_test, auc_avg_train = auc_test/10, auc_train/10
    accuracy_avg_test, accuracy_avg_train = accuracy_test/10, accuracy_train/10
    auc_avg_test_dict[key]=auc_avg_test
    auc_avg_train_dict[key]=auc_avg_train
    accuracy_avg_test_dict[key]=accuracy_avg_test
    accuracy_avg_train_dict[key]=accuracy_avg_train
    sensi_avg_test, sensi_avg_train = sensi_test/10, sensi_train/10
    speci_avg_test, speci_avg_train = speci_test/10, speci_train/10
    sensi_avg_test_dict[key]=sensi_avg_test
    sensi_avg_train_dict[key]=sensi_avg_train
    speci_avg_test_dict[key]=speci_avg_test
    speci_avg_train_dict[key]=speci_avg_train
print(auc_avg_test_dict)
print(accuracy_avg_test_dict)
print(sensi_avg_test_dict)
print(speci_avg_test_dict)

{'feature select time > 0': 0.522638888888889, 'feature select time > 1': 0.6322222222222221, 'feature select time > 2': 0.5931944444444444, 'feature select time > 3': 0.5345833333333334, 'feature select time > 4': 0.5345833333333334}
{'feature select time > 0': 0.5201754385964913, 'feature select time > 1': 0.6342105263157894, 'feature select time > 2': 0.5964912280701754, 'feature select time > 3': 0.5383040935672514, 'feature select time > 4': 0.5383040935672514}
{'feature select time > 0': 0.5652777777777778, 'feature select time > 1': 0.6, 'feature select time > 2': 0.5430555555555556, 'feature select time > 3': 0.49027777777777776, 'feature select time > 4': 0.49027777777777776}
{'feature select time > 0': 0.4800000000000001, 'feature select time > 1': 0.6644444444444445, 'feature select time > 2': 0.6433333333333333, 'feature select time > 3': 0.5788888888888888, 'feature select time > 4': 0.5788888888888888}


In [51]:
print(auc_avg_train_dict)
print(accuracy_avg_train_dict)
print(sensi_avg_train_dict)
print(speci_avg_train_dict)

{'feature select time > 0': 0.6356412510828668, 'feature select time > 1': 0.652589603185891, 'feature select time > 2': 0.6133601517952961, 'feature select time > 3': 0.5493436049443374, 'feature select time > 4': 0.5493436049443374}
{'feature select time > 0': 0.6343848018249216, 'feature select time > 1': 0.6535286569717706, 'feature select time > 2': 0.616463501568292, 'feature select time > 3': 0.551960364984317, 'feature select time > 4': 0.551960364984317}
{'feature select time > 0': 0.6590506329113923, 'feature select time > 1': 0.6350949367088606, 'feature select time > 2': 0.5555379746835443, 'feature select time > 3': 0.5012025316455696, 'feature select time > 4': 0.5012025316455696}
{'feature select time > 0': 0.6122318692543413, 'feature select time > 1': 0.6700842696629213, 'feature select time > 2': 0.671182328907048, 'feature select time > 3': 0.5974846782431052, 'feature select time > 4': 0.5974846782431052}


In [ ]:
#---------DONE--------------